In [1]:
!nvidia-smi

Fri Feb  7 07:30:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.77                 Driver Version: 565.77         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L40                     Off |   00000000:A8:00.0 Off |                    0 |
| N/A   34C    P0             78W /  300W |   43965MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Load libraries, define constants, functions, and classes

* libraries

In [2]:
import os

import sys
sys.path.append("../2_train_and_test_models")

import numpy as np
import pandas as pd

from params import ROOT
from collections import defaultdict
from sklearn.metrics import average_precision_score, roc_auc_score

* paths and constants

In [3]:
os.makedirs(f"{ROOT}/plots", exist_ok=True)

In [4]:
# Shorthand names and plot 'acceptable' names
SPECIES = ["mm10", "hg38"]
TFS     = ["CTCF", "CEBPA", "HNF4A", "RXRA"]
RUNS    = 5

# Shorthand names for all model types to include in plots
MODELS = [
    "BM",
    "GRL",
    "MORALE"
]

# Plot-acceptable names for model types
MODEL_NAMES = {
    "BM-mm10": "Mouse-trained",
    "BM-hg38": "Human-trained",
    "GRL-mm10": "Mouse-trained (+GRL)",
    "GRL-hg38": "Human-trained (+GRL)",
    "MORALE-mm10": "Mouse-trained (+MORALE)",
    "MORALE-hg38": "Human-trained (+MORALE)"
}

* Helper functions we use

In [5]:
def get_preds_file(model_suffix, tf, train_species, test_species):
    preds_root = f"{ROOT}/output"
    return f"{preds_root}/{model_suffix}_tf-{tf}_trained-{train_species}_tested-{test_species}.preds.npy"

def get_labels_file(model_suffix, tf, train_species, test_species):
    labels_root = f"{ROOT}/output"
    return f"{labels_root}/{model_suffix}_tf-{tf}_trained-{train_species}_tested-{test_species}.labels.npy"

def load_all_test_sets():
    preds_dict  = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(dict))))
    labels_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(dict))))

    # Loop over mouse-trained, human-trained models, and domain-adaptive models
    for model in MODELS:
        for specie in SPECIES:
            model_name = f"{model}-{specie}"
            for tf in TFS:
                for domain in SPECIES:
                    print("=== " + tf + ", " + model_name + f"-trained, {domain}-tested ===")
                    preds_file  = get_preds_file(model_suffix=model, tf=tf, train_species=specie, test_species=domain)
                    labels_file = get_labels_file(model_suffix=model, tf=tf, train_species=specie, test_species=domain)
                    preds_dict[model_name][tf][domain]   = np.load(preds_file)
                    labels_dict[model_name][tf][domain]  = np.load(labels_file)

    return preds_dict, labels_dict

In [6]:
def format_data_for_seaborn(auPRC_dicts, auROC_dicts):
    # This function re-formats the performance dictionaries into
    # one pandas DataFrame that matches how seaborn expects.
    tf_col      = []
    domain_col  = []
    model_col   = []
    auprc_col   = []
    auroc_col   = []

    model_list  = list(auPRC_dicts.keys())
    reps        = len(auPRC_dicts[model_list[0]][TFS[0]]['mm10']) # We just take mm10 here

    for model in MODELS:
        for specie in SPECIES:
            model_name = f"{model}-{specie}"
            for tf in TFS:
                for domain in SPECIES:
                    # We extend these based on the entires from the five-fold cross validation
                    model_col.extend([MODEL_NAMES[model_name]] * reps)
                    tf_col.extend([tf] * reps)
                    domain_col.extend([domain] * reps)

                    # Now grab the five-fold for the current TF and domain
                    auprc_col.extend(auPRC_dicts[model_name][tf][domain])
                    auroc_col.extend(auROC_dicts[model_name][tf][domain])
        
    return pd.DataFrame({"TF":tf_col, "Model":model_col, "Eval": domain_col, "auPRC":auprc_col, "auROC":auroc_col})

def get_auPRCs(labels, preds):
    # This function calculates the auPRC for each set of
    # predictions passed in. The length of the 2nd axis
    # of the predictions array passed in will be the # of
    # auPRCs returned as a list. The length of the 1st axis
    # of the predictions array should match the length
    # of the labels array.        

    # (1) Reframe it into an array
    preds  = [np.squeeze(preds[:, run]) for run in range(RUNS)]

    # (2) Adjust the label length to match the prediction length
    lens_to_truncate_by = [pred.shape[0] for pred in preds]
    
    # Assert all label lengths in list are the same
    assert len(set(lens_to_truncate_by)) == 1, "All label lengths are the same"

    labels = labels[:lens_to_truncate_by[0]]

    # (3) Calculate the AUC-ROC and AUC-PR for each of the five-folds
    return [average_precision_score(labels, pred) for pred in preds]

def get_auROCs(labels, preds):
    # This function calculates the auROC for each set of
    # predictions passed in. The length of the 2nd axis
    # of the predictions array passed in will be the # of
    # auROCs returned as a list. The length of the 1st axis
    # of the predictions array should match the length
    # of the labels array.

    # (1) Reframe it into an array
    preds  = [np.squeeze(preds[:, run]) for run in range(RUNS)]

    # (2) Adjust the label length to match the prediction length
    lens_to_truncate_by = [pred.shape[0] for pred in preds]

    # Assert all label lengths in list are the same
    assert len(set(lens_to_truncate_by)) == 1, "All label lengths are the same"

    labels = labels[:lens_to_truncate_by[0]]

    # (3) Calculate the AUC-ROC and AUC-PR for each of the five-folds
    return [roc_auc_score(labels, pred) for pred in preds]

def get_performance_df(preds_dict, labels_dict):
    # This function loads in binding labels for each TF for 
    # a given test species, and for each TF, calculates the auPRC
    # using each set of predictions that is input in "preds_dict".
    auPRC_dicts = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(dict))))
    auROC_dicts = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(dict))))

    for model in MODELS:
        for specie in SPECIES:
            model_name = f"{model}-{specie}"
            for tf in TFS:
                for domain in SPECIES:
                    model_preds = preds_dict[model_name][tf][domain]
                    labels      = labels_dict[model_name][tf][domain]
                    
                    # Save predictions
                    auPRC_dicts[model_name][tf][domain] = get_auPRCs(labels, model_preds)
                    auROC_dicts[model_name][tf][domain] = get_auROCs(labels, model_preds)

    # Before returning all the auPRCs in dictionaries,
    # we just need to reformat how they are stored
    # because seaborn expects particularly formatted input
    return format_data_for_seaborn(auPRC_dicts, auROC_dicts)

# Save into usable format

In [7]:
preds, labels   = load_all_test_sets()
performance_df  = get_performance_df(preds, labels)

=== CTCF, BM-mm10-trained, mm10-tested ===
=== CTCF, BM-mm10-trained, hg38-tested ===
=== CEBPA, BM-mm10-trained, mm10-tested ===
=== CEBPA, BM-mm10-trained, hg38-tested ===
=== HNF4A, BM-mm10-trained, mm10-tested ===
=== HNF4A, BM-mm10-trained, hg38-tested ===
=== RXRA, BM-mm10-trained, mm10-tested ===
=== RXRA, BM-mm10-trained, hg38-tested ===
=== CTCF, BM-hg38-trained, mm10-tested ===
=== CTCF, BM-hg38-trained, hg38-tested ===
=== CEBPA, BM-hg38-trained, mm10-tested ===
=== CEBPA, BM-hg38-trained, hg38-tested ===
=== HNF4A, BM-hg38-trained, mm10-tested ===
=== HNF4A, BM-hg38-trained, hg38-tested ===
=== RXRA, BM-hg38-trained, mm10-tested ===
=== RXRA, BM-hg38-trained, hg38-tested ===
=== CTCF, GRL-mm10-trained, mm10-tested ===
=== CTCF, GRL-mm10-trained, hg38-tested ===
=== CEBPA, GRL-mm10-trained, mm10-tested ===
=== CEBPA, GRL-mm10-trained, hg38-tested ===
=== HNF4A, GRL-mm10-trained, mm10-tested ===
=== HNF4A, GRL-mm10-trained, hg38-tested ===
=== RXRA, GRL-mm10-trained, mm10-tes

In [8]:
print(performance_df.to_string())

        TF                    Model  Eval     auPRC     auROC
0     CTCF            Mouse-trained  mm10  0.733746  0.982818
1     CTCF            Mouse-trained  mm10  0.738238  0.985658
2     CTCF            Mouse-trained  mm10  0.725653  0.980432
3     CTCF            Mouse-trained  mm10  0.732232  0.983752
4     CTCF            Mouse-trained  mm10  0.723825  0.982458
5     CTCF            Mouse-trained  hg38  0.572272  0.954654
6     CTCF            Mouse-trained  hg38  0.581242  0.964336
7     CTCF            Mouse-trained  hg38  0.584048  0.954533
8     CTCF            Mouse-trained  hg38  0.585125  0.957292
9     CTCF            Mouse-trained  hg38  0.581613  0.955849
10   CEBPA            Mouse-trained  mm10  0.347946  0.944431
11   CEBPA            Mouse-trained  mm10  0.349121  0.944012
12   CEBPA            Mouse-trained  mm10  0.339244  0.944986
13   CEBPA            Mouse-trained  mm10  0.344483  0.940298
14   CEBPA            Mouse-trained  mm10  0.351585  0.943639
15   CEB

## Now export

In [9]:
performance_df.to_csv(ROOT + f"/plots/performance_data.csv")

-----